In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')
from datetime import datetime, timedelta

import keras
import tensorflow

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Any results you write to the current directory are saved as output.
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
#load the data
data = pd.read_csv('energy_dataset.csv',
                   index_col=[0],
                   parse_dates=True)

dates = pd.date_range(start='2014-12-31T2300', end='2018-12-31T2200', freq='H')
data.index = pd.DatetimeIndex(dates).tz_localize('UTC').tz_convert('Europe/Madrid')
df = data[['total load actual', 'total load forecast']]

null_vals = df.isnull().sum()
print('Null values in the target column {}'.format(null_vals))
df.head()
df.dropna(inplace=True)

Null values in the target column total load actual      36
total load forecast     0
dtype: int64


C:\Users\Ajay\AppData\Local\Temp\ipykernel_13600\2583871192.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [3]:
data=df.filter(['total load actual'])[0:1000]
dataset=data.values

In [7]:
# univariate stacked lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# split a univariate sequence
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=2)


Epoch 1/200
1/1 - 2s - loss: 4453.5493 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 4424.7368 - 6ms/epoch - 6ms/step
Epoch 3/200
1/1 - 0s - loss: 4393.2637 - 5ms/epoch - 5ms/step
Epoch 4/200
1/1 - 0s - loss: 4358.4390 - 5ms/epoch - 5ms/step
Epoch 5/200
1/1 - 0s - loss: 4318.9863 - 5ms/epoch - 5ms/step
Epoch 6/200
1/1 - 0s - loss: 4274.1455 - 10ms/epoch - 10ms/step
Epoch 7/200
1/1 - 0s - loss: 4223.3521 - 6ms/epoch - 6ms/step
Epoch 8/200
1/1 - 0s - loss: 4166.0781 - 6ms/epoch - 6ms/step
Epoch 9/200
1/1 - 0s - loss: 4102.9854 - 6ms/epoch - 6ms/step
Epoch 10/200
1/1 - 0s - loss: 4035.5293 - 5ms/epoch - 5ms/step
Epoch 11/200
1/1 - 0s - loss: 3965.8125 - 0s/epoch - 0s/step
Epoch 12/200
1/1 - 0s - loss: 3894.3391 - 6ms/epoch - 6ms/step
Epoch 13/200
1/1 - 0s - loss: 3819.6992 - 6ms/epoch - 6ms/step
Epoch 14/200
1/1 - 0s - loss: 3740.5769 - 6ms/epoch - 6ms/step
Epoch 15/200
1/1 - 0s - loss: 3654.1111 - 10ms/epoch - 10ms/step
Epoch 16/200
1/1 - 0s - loss: 3557.5369 - 5ms/epoch - 5ms/step
E

1/1 - 0s - loss: 1.7995 - 8ms/epoch - 8ms/step
Epoch 135/200
1/1 - 0s - loss: 1.7888 - 5ms/epoch - 5ms/step
Epoch 136/200
1/1 - 0s - loss: 1.7780 - 8ms/epoch - 8ms/step
Epoch 137/200
1/1 - 0s - loss: 1.7665 - 5ms/epoch - 5ms/step
Epoch 138/200
1/1 - 0s - loss: 1.7544 - 8ms/epoch - 8ms/step
Epoch 139/200
1/1 - 0s - loss: 1.7423 - 5ms/epoch - 5ms/step
Epoch 140/200
1/1 - 0s - loss: 1.7306 - 8ms/epoch - 8ms/step
Epoch 141/200
1/1 - 0s - loss: 1.7195 - 6ms/epoch - 6ms/step
Epoch 142/200
1/1 - 0s - loss: 1.7088 - 8ms/epoch - 8ms/step
Epoch 143/200
1/1 - 0s - loss: 1.6982 - 6ms/epoch - 6ms/step
Epoch 144/200
1/1 - 0s - loss: 1.6872 - 7ms/epoch - 7ms/step
Epoch 145/200
1/1 - 0s - loss: 1.6755 - 2ms/epoch - 2ms/step
Epoch 146/200
1/1 - 0s - loss: 1.6635 - 13ms/epoch - 13ms/step
Epoch 147/200
1/1 - 0s - loss: 1.6514 - 5ms/epoch - 5ms/step
Epoch 148/200
1/1 - 0s - loss: 1.6397 - 8ms/epoch - 8ms/step
Epoch 149/200
1/1 - 0s - loss: 1.6287 - 6ms/epoch - 6ms/step
Epoch 150/200
1/1 - 0s - loss: 1.618

In [10]:
# demonstrate prediction
x_input = array([80, 90, 100])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[116.57713]]
